<a href="https://colab.research.google.com/github/Elzawawy/covid-case-estimator/blob/master/Worldometer_Daily_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports cell 
from shutil import copyfile
import pandas as pd
import numpy as np
import pickle

# mount google drive to copy files from repo into drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### Clone Repoistory to Temporary Drive and Sync latest data with the Storage directory on permanent google drive storage.

In [3]:
# download repoistory from Github and Sync with Drive data.
!git clone https://github.com/bumbeishvili/covid19-daily-data
REPO_DIR = "/content/covid19-daily-data/"
STORAGE_DIR = "/content/drive/My Drive/COVID-19/worldometer-daily-data/"
CONFIRMED_FILE = "time_series_19-covid-Confirmed.csv"
RECOVERED_FILE = "time_series_19-covid-Recovered.csv"
DEATHS_FILE = "time_series_19-covid-Deaths.csv"

# copy files from temproary repo directory to permanent drive directory.
copyfile(REPO_DIR+CONFIRMED_FILE, STORAGE_DIR+CONFIRMED_FILE);
copyfile(REPO_DIR+RECOVERED_FILE, STORAGE_DIR+RECOVERED_FILE);
copyfile(REPO_DIR+DEATHS_FILE, STORAGE_DIR+DEATHS_FILE);

fatal: destination path 'covid19-daily-data' already exists and is not an empty directory.


## Let's Begin by understanding the Daily Confirmed Cases Table. 

*   Originally, it has 4 main columns **(Province/State - Country/Region - Long - Lat)** and more variable columns based on days. 

* Each day constitues a column, thus since each day this dataset gets updated we basically get a new column added each day.

* The dataset starts at day **1/22/2020** which when Corona Vrius started appearing for the first time in China and ends at the day you will run this Notebook at while the Github Repoistory keeps updating each day.

In [4]:
confirmed_cases_dataframe = pd.read_csv(STORAGE_DIR+CONFIRMED_FILE)
confirmed_cases_dataframe.head()

,Province/State,Country/Region,Long,Lat,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20
0,Afghanistan,Afghanistan,65.0000,33.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,349,367,423,444,484,521,555,607,665,714,784,840,906,933,996,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894,3224,3392,3563,3778,4033,4402,4687,4963,5226,5639
1,Albania,Albania,20.1683,41.1533,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,361,377,383,400,409,416,433,446,467,475,494,518,539,548,562,584,609,634,663,678,712,726,736,750,766,773,782,789,795,803,820,832,842,850,856,868,872,876,880,898
2,Algeria,Algeria,1.6596,28.0339,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,1320,1423,1468,1572,1666,1761,1825,1914,1983,2070,2160,2268,2418,2534,2629,2718,2811,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154,4295,4474,4648,4838,4997,5182,5369,5558,5723,5891,6067,6253,6442
3,Andorra,Andorra,1.5218,42.5063,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,501,525,545,564,583,601,601,638,646,659,673,673,696,704,713,717,717,723,723,731,738,738,743,743,743,745,745,747,748,750,751,751,752,752,754,755,755,758,760,761
4,Angola,Angola,17.8739,-11.2027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,14,16,17,19,19,19,19,19,19,19,19,19,19,24,24,24,24,25,25,25,25,26,27,27,27,27,30,35,35,35,36,36,36,43,43,43,45,45,45,48


### What we need to do to this dataset to match our needs ?


*   We can drop the (Long - Lat - Province/State) columns since no need for them for us. **Why ?**
  * Longitude and lattiude is something that probably won't affect the coronavirus spead. 
  * We are only interested in countries not states inside them.

*  We will need to separate a dataset **for each country** with days as row instances and the cell values are the number of daily cases at this country **(1-D arrays: number of cases)**. Date of the day can be implicit itself from the bigger dataset.

* We will do this for each country and put them in a **Map (dict) structure** with country as key and it's 1-D array as value.


In [0]:
def create_cases_dict(cases_dataframe):
  cases_dataframe = cases_dataframe.drop(columns=["Long","Lat","Province/State"])
  country_cases = {}
  # iterate on each row, put the Country Name in key and rest of day values as 1-D array value.
  for row in cases_dataframe.itertuples(index=False):
    country_cases[row[0]] = np.array([day for day in row[1:]])
  return country_cases

def save_dict_to_pickle(dict, pickle_file):
  with open(pickle_file, 'wb') as handle:
    pickle.dump(dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
confirmed_cases_dict = create_cases_dict(confirmed_cases_dataframe)
save_dict_to_pickle(confirmed_cases_dict, STORAGE_DIR+"confirmed_cases_dict.pickle")

### We now repeat the same steps for Recovered Cases.

In [0]:
recovered_cases_dataframe = pd.read_csv(STORAGE_DIR+RECOVERED_FILE)
recovered_cases_dict = create_cases_dict(recovered_cases_dataframe)
save_dict_to_pickle(recovered_cases_dict, STORAGE_DIR+"recovered_cases_dict.pickle")

### Lastly, repeat the same steps for Death Cases.

In [0]:
deaths_cases_dataframe = pd.read_csv(STORAGE_DIR+DEATHS_FILE)
deaths_cases_dict = create_cases_dict(deaths_cases_dataframe)
save_dict_to_pickle(deaths_cases_dict, STORAGE_DIR+"deaths_cases_dict.pickle")